# Vascular Functionality Battery - Main Notebook

## 1. Import libraries

In [1]:
import subprocess, shutil, glob
import os, sys, pathlib
mymodules = os.fspath(pathlib.Path.cwd().joinpath('../CustomModules'))
if mymodules not in sys.path:
    sys.path.append(mymodules)
import numpy as np
import pandas as pd
import MRIhelperfunctions as h
import calciumGroupFunctions as g
import GUIfunctions as gui
from MRIhelperfunctions import Settings
from imp import importlib
import papermill as pm
import matplotlib.pyplot as plt

import ipywidgets as widgets

%matplotlib inline

In [4]:
h = importlib.reload(h)
g = importlib.reload(g)
gui = importlib.reload(gui)
# from MRIhelperfunctions import Settings

## 2. Settings
Enter information and select desired analyses

In [5]:
s, w = gui.MainGUI(mainfolder='/mnt/c/Users/felix/Documents/exampledata_python')

In [3]:
gui.update_shelve(s, w)

In [6]:
main_folder = '/home/aic/BigHDD/MR_DATA/GLUT1'   #<-- change this
raw_folder = os.path.join(main_folder,'Rawdata')

excel_filename = 'Animal_Scan_IDs.xlsx'  #<-- change this. Must be placed inside main folder


analysis_folder = os.path.join(main_folder,'Analysis')
notebookfolder = '/BigHDD/MR_DATA/GithubRepositories/CalciumfMRI/fMRI_notebooks'

folders = {}
folders['epi_roi'] = '/home/aic/BigHDD/MR_DATA/GLUT1/Templates/ROIs_EPI.nii'


folders.update({'main':main_folder, 'raw':raw_folder, 'analysis':analysis_folder, 'notebooks':notebookfolder, 'excel':excel_filename})
# stores dataframes to plot for later
plots={}

# Check in output below if Excel file is correctly recognized
info = h.getinfo(folders)
info

,glut1_male1,glut1_male2,glut1_f10wks1,glut1_f10wks2,glut1_f10wks3,glut1_f10wks4,glut1_f10wks5,glut1_f10wks6,glut1_f8wks1,glut1_f8wks2,glut1_f8wks3,glut1_f8wks4,glut1_f8wks5,glut1_f8wks6,glut1_f8wks7,glut1_f6wks2,glut1_f6wks1
scan,,,,,,,,,,,,,,,,,
genotype,1,1,1,2,1,2,2,1,1,1,2,2,2,2,1,1,1
Gd DCE,6,8,6,5,5,8,5,5,7,5,6,6,8,8,6,5,5
Angio pre,8,9,7,6,6,9,6,6,8,6,9,8,9,9,8,6,6
Angio post,12,12,11,9,9,13,10,11,12,11,12,12,16,13,12,9,10
Endorem DCE,11,11,10,8,8,12,9,10,10,10,11,11,15,12,11,8,9
Diamox,15,14,13,11,13,15,12,13,15,13,16,17,20,16,14,11,13
folder,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...


In [9]:
# use code below to locate folders of individual animals:
info.loc['folder','glut1_f10wks1']

'/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/20180607_084714_Glut1_2_4'

## 3. Main loop

Runs the selected analyses (modules) for each individual animal.

- To turn a certain analysis on or off: comment/uncomment the line with *#*
- to run the loop only on a single/subset of animals: index *info.columns*, e.g. info.columns[1:2] would run only the 2nd animal
- If the analysis crashes after a couple of animals, use the option 'overwrite = False' to run loop again only on the animals that were not yet processed.


In [10]:
for animal in ['glut1_f8wks4']:  # to run only one or subset of animals, enter here and comment out line below.
# for animal in info.columns:   
    print('animal: ',animal)
    folders['animal']=info.loc['folder',animal]
    inputs = dict(folders=folders, animal=animal, info=info.to_json())
    
    
    h.run_module(folders, 'Angiography', inputs)  # ~2-3 min / animal

#     h.run_module(folders, 'Leakage', inputs, overwrite = True) # ~10-15 min / animal

#     h.run_module(folders, 'BolusTracking', inputs) # ~8 min / animal

#     h.run_module(folders, 'Diamox', inputs, overwrite = False) # ~20-50 min / animal

print('All complete!')


animal:  glut1_f8wks4
running Angiography analysis...


Input Notebook:  /BigHDD/MR_DATA/GithubRepositories/CalciumfMRI/fMRI_notebooks/Angiography.ipynb
Output Notebook: /home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/20180614_112934_Glut1_2_13/Angiography_output.ipynb
100%|██████████| 23/23 [03:00<00:00,  7.83s/it]


All complete!


## 4. Collecting outputs for group analysis

copies only relevant files of each animal in group folder

In [34]:
folders['group'] = h.folder_check_create(folders['main'], 'Groupstats')


for animal in info.columns:   
    folders['animal']=info.loc['folder',animal]

#     info, folders = h.copy_notebook_outputs(folders, animal, 'Angiography', info)

    info, folders = h.copy_notebook_outputs(folders, animal, 'BolusTracking', info)
    
#     info, folders = h.copy_notebook_outputs(folders, animal, 'Diamox', info)
    
#     info, folders = h.copy_notebook_outputs(folders, animal, 'Leakage', info)
    
    
    # copying some underlay files to the folders
    template_file = os.path.join(folders['main'], 'Templates', 'BolusTracking.nii')    
    output = os.path.join(folders['group'], 'BolusTracking', 'Template.nii')
    shutil.copy(template_file, output)
    output = os.path.join(folders['group'], 'Diamox', 'Template.nii') # same scan settings as BolusTracking
    shutil.copy(template_file, output)
    
info

,glut1_male1,glut1_male2,glut1_f10wks1,glut1_f10wks2,glut1_f10wks3,glut1_f10wks4,glut1_f10wks5,glut1_f10wks6,glut1_f8wks1,glut1_f8wks2,glut1_f8wks3,glut1_f8wks4,glut1_f8wks5,glut1_f8wks6,glut1_f8wks7,glut1_f6wks2,glut1_f6wks1
scan,,,,,,,,,,,,,,,,,
genotype,1,1,1,2,1,2,2,1,1,1,2,2,2,2,1,1,1
Gd DCE,6,8,6,5,5,8,5,5,7,5,6,6,8,8,6,5,5
Angio pre,8,9,7,6,6,9,6,6,8,6,9,8,9,9,8,6,6
Angio post,12,12,11,9,9,13,10,11,12,11,12,12,16,13,12,9,10
Endorem DCE,11,11,10,8,8,12,9,10,10,10,11,11,15,12,11,8,9
Diamox,15,14,13,11,13,15,12,13,15,13,16,17,20,16,14,11,13
folder,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...
diamox_slope,NaN,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...
diamox_ttp,NaN,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...


In [4]:
# to display location of individual files (first column name, then animal name)
info.loc['folder','glut1_f10wks2']

'/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/20180607_113110_Glut1_2_5'

## 5. Create average & statistical maps

In [13]:
# general functions and settings
# extracting ROI time series, if ROI file above exists.
def name_rois(group1, group2, roipath):
    if os.path.exists(roipath):
        for filepath in group1:
            filename = os.path.splitext(os.path.basename(filepath))[0] #remove .nii extension
            roiname = f"{filename}.group1"
            # e.g. diamox_max_glut1_f10wks5.group1
            h.extract_roi(filepath, roipath, name=roiname)
        for filepath in group2:
            filename = os.path.splitext(os.path.basename(filepath))[0]
            roiname = f"{filename}.group2"
            h.extract_roi(filepath, roipath, name=roiname)

### 6. a) Leakage - t-test

In [14]:
map_names = ['Leakage_max']   # prefixes of the output files

for map_type in map_names:
    group1, group2 = h.select_for_group_average(folders['group_Leakage'],
                             info, map_type,
                             exclude='male',  #<-- excludes animals that contain this string in animal name
                             row = 'genotype')
    h.ttest_2groups(folders['group_Leakage'], map_type, group1, group2, blur=0.0)

KeyError: 'group_Leakage'

### 6. b) Bolus Tracking t-test

In [15]:
map_names = ['CBF_map', 'CBV_bolus_map', 'MTT_map', 'CBV_steadystate_map']   # prefixes of the output files
to_exclude = ['male']

for map_type in map_names:
    group1, group2 = h.select_for_group_average(folders['group_BolusTracking'],
                             info, map_type,
                             exclude=to_exclude,  #<-- excludes animals that contain this string in animal name
                             row = 'genotype')
    h.ttest_2groups(folders['group_BolusTracking'], map_type, group1, group2, blur=0.4)
    
    name_rois(group1, group2, folders['epi_roi'])

# to get the spreadsheet with all ROI averages
df, plots['bolustracking'] = h.create_clean_csv(folders['group_BolusTracking'])

KeyError: 'group_BolusTracking'

### 6. c) Diamox t-test

In [12]:
map_names = ['diamox_ttp', 'diamox_slope', 'diamox_max']
to_exclude = ['male','f8wks3'] #ghosting artifact


for map_type in map_names:
    group1, group2 = h.select_for_group_average(folders['group_Diamox'],
                             info, map_type,
                             exclude=to_exclude,  #<-- excludes animals that contain this string in animal name
                             row = 'genotype')
    h.ttest_2groups(folders['group_Diamox'], map_type, group1, group2, blur=0.3)
    
    name_rois(group1, group2, folders['epi_roi']) # same ROI, because its EPI with same geometry. 
    
df, plots['diamox']  = h.create_clean_csv(folders['group_Diamox'],'diamox')

KeyError: 'group_Diamox'

## 7. Plot group differences

In [11]:
# make plots pop up in new window
%matplotlib qt

# h.MR_boxplots(pd.read_csv(plots['bolustracking']))

h.MR_boxplots(pd.read_csv(plots['diamox']))

KeyError: 'diamox'